# AZ-204: Develop message-based solutions

In [1]:
import sys
sys.path.append('./../utils')
import ai

## Discover Azure message queues

In [3]:
%%learn
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/

https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/1-introduction
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/2-choose-queue-solution
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/3-azure-service-bus-overview
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/4-queues-topics-subscriptions
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/5-messages-payloads-serialization
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/6-send-receive-messages-service-bus
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/7-azure-queue-storage-overview
https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/8-queue-storage-code-examples


('',
 'https://learn.microsoft.com/en-us/training/modules/discover-azure-message-queue/\n')

In [4]:
%%question
What is the purpose of queuing (in the context of an enterprise application)?|

In the context of an **enterprise application**, the purpose of **queuing** is to ensure **reliable** and **efficient** handling of **asynchronous** tasks. It helps manage **workloads**, balance system **performance**, and maintain **scalability** by processing tasks in a **controlled** manner.

In [6]:
%%question
Provide an example of a scenario where you would use a queue mechanism in an enterprise LOB application?

In an enterprise **Line of Business (LOB)** application, a **queue mechanism** can be used in **order processing**. When customers place orders, each order is added to a queue. This allows the system to process each order sequentially, ensuring that inventory checks, payment processing, and order fulfillment are handled efficiently and without errors, even during peak times.

In [7]:
%%question
What are the two types of queue mechanisms that Azure Support?

Azure supports two types of queue mechanisms:

1. **Azure Storage Queues**: Part of Azure Storage, designed for **simple** message queuing, offering **reliable** and **persistent** messaging.

2. **Azure Service Bus Queues**: Part of the Service Bus, designed for more **complex** messaging needs, supporting **advanced** features like **dead-lettering**, **transactions**, and **topic-based publish/subscribe**.

In [6]:
%%question
Can you compare those two types of queue mechanisms in table format?

Certainly! Here's a comparison of **Service Bus queues** and **Storage queues** in Azure:

| Feature | **Service Bus Queues** | **Storage Queues** |
| --- | --- | --- |
| Infrastructure | Part of Azure messaging infrastructure | Part of Azure Storage infrastructure |
| Message Size | Up to 100 MB | Up to 64 KB |
| FIFO Support | Guaranteed FIFO | No strict FIFO guarantee |
| Message Retention | Supports message deferral | Messages can be stored for up to 7 days (or longer with updates) |
| Protocols | Supports AMQP and HTTP/REST | Supports HTTP/HTTPS |
| Advanced Features | Supports transactions, sessions, duplicate detection | Basic queuing features |
| Access Control | Role-based access control | Simpler access model |
| Use Case | Suitable for complex messaging patterns | Suitable for simple queue storage with large message volumes |

Each mechanism is suited for different scenarios based on the complexity and requirements of the application.

In [8]:
%%question
Provide a C# example of using Azure Storage Queues (both sending and receiving messages)

Here's an example of how to use Azure Storage Queues in C# to send and receive messages:

### Prerequisites:
- Install Azure.Storage.Queues NuGet package.

### Code Example:

```csharp
using System;
using System.Threading.Tasks;
using Azure.Storage.Queues; // Namespace for Queue storage types
using Azure.Storage.Queues.Models; // Namespace for QueueMessage

class Program
{
    private const string connectionString = "<Your Azure Storage Connection String>";
    private const string queueName = "myqueue";

    static async Task Main(string[] args)
    {
        // Create a queue client
        QueueClient queueClient = new QueueClient(connectionString, queueName);

        // Create the queue if it doesn't already exist
        await queueClient.CreateIfNotExistsAsync();

        if (queueClient.Exists())
        {
            Console.WriteLine($"Queue '{queueName}' created.");

            // Send a message to the queue
            string message = "Hello, Azure Queue!";
            await queueClient.SendMessageAsync(message);
            Console.WriteLine($"Sent: {message}");

            // Receive a message from the queue
            QueueMessage[] messages = await queueClient.ReceiveMessagesAsync(maxMessages: 1);
            if (messages.Length > 0)
            {
                Console.WriteLine($"Received: {messages[0].MessageText}");

                // Delete the message from the queue
                await queueClient.DeleteMessageAsync(messages[0].MessageId, messages[0].PopReceipt);
                Console.WriteLine("Message deleted.");
            }
        }
    }
}
```

### Steps:
1. Replace `"<Your Azure Storage Connection String>"` with your actual Azure Storage account connection string.
2. The program creates a queue if it doesn't exist, sends a message, receives it, and then deletes it from the queue.
3. Run the program, and you should see the message being sent and received.

This example demonstrates basic operations with Azure Storage Queues using the Azure.Storage.Queues library in C#.

In [9]:
%%question
What are the different tiers of Service Bus? Explain in table format highlighting the differences.

Here’s a table highlighting the differences between the different tiers of **Azure Service Bus**:

| Feature | **Basic** | **Standard** | **Premium** |
| --- | --- | --- | --- |
| Throughput | Low | Variable | High |
| Performance | Variable | Variable | Predictable |
| Pricing | Lower, fixed rate | Pay as you go | Fixed pricing |
| Features | Basic messaging | Topics, subscriptions, standard broker features | Advanced features like isolation, Geo-disaster recovery, and more |
| Message Size | Up to 256 KB | Up to 256 KB | Up to 100 MB |
| Scale | Limited | N/A | Scalable workloads |
| Use Case | Simple applications, hobby projects | Medium complexity applications | Mission-critical enterprise applications |

Each tier is designed to cater to different use cases, from simple applications using the Basic tier to mission-critical, high-performance applications using the Premium tier.

In [8]:
%%question
When receiving messages from Service Bus, there is receive and delete or peek lock. What is this?

In Azure **Service Bus**, the two message reception modes are:

1. **Receive and Delete**: Messages are **removed** from the queue as soon as they are received. This is a **simpler** approach but risks losing messages if processing fails after receipt.

2. **Peek Lock**: Messages are **locked** for processing but remain in the queue. This allows for **safe** processing; if processing fails, the message can be **unlocked** and made available to other receivers.

In [11]:
%%question
What is duplicate detection and how does it work in Service Bus?

**Duplicate detection** in Azure **Service Bus** is a feature that helps prevent the processing of duplicate messages, ensuring that each message is processed only once. 

### How it Works:
- **Message Identification**: Each message sent to the Service Bus is assigned a unique **MessageId** by the sender.
- **Detection Window**: Service Bus maintains a history of MessageIds for a specified **duplicate detection time window** (up to 7 days).
- **Processing**: If a message with the same MessageId is sent within this time window, Service Bus identifies it as a duplicate and does not deliver it to the receiver.
- **Configuration**: Duplicate detection is an optional feature that can be enabled when creating a queue or topic, and the time window for detection can be configured.

This feature is particularly useful in scenarios where network issues or errors might cause a sender to unintentionally send the same message multiple times.

In [9]:
%%question
Service Bus has a feature called dead-lettering. What is this?

**Dead-lettering** in Azure **Service Bus** is a feature that helps handle messages that cannot be **processed** successfully. When a message cannot be delivered or processed due to errors or exceeds a defined **retry** count, it is moved to a **dead-letter queue**. This allows for **separate analysis** and handling of problematic messages without affecting the main queue.

In [15]:
%%question
Will I get notified if a message is dead-lettered in Service Bus?

Azure Service Bus does not provide direct notifications when a message is dead-lettered. However, you can implement a monitoring mechanism to handle this:

1. **Polling**: Regularly poll the dead-letter queue to check for new messages.
2. **Azure Monitor**: Use Azure Monitor and logging to track message processing and identify when messages are moved to the dead-letter queue.
3. **Custom Logic**: Implement custom application logic to send alerts or notifications when a message is dead-lettered.

These methods help ensure you are aware of and can respond to messages that have been dead-lettered.

In [13]:
%%question
Is this also available in Azure Storage Queues?

No, **Azure Storage Queues** do not have a built-in **dead-letter queue** feature like Azure Service Bus. However, you can implement a similar mechanism manually by:

1. **Tracking Message Processing**: Monitor message processing attempts and move messages that fail repeatedly to a separate queue designated as a dead-letter queue.
2. **Custom Logic**: Implement custom logic in your application to handle and analyze undeliverable messages.

This manual approach can help you manage problematic messages in Azure Storage Queues.

In [10]:
%%question
Service Bus can handle messages with a session. What is this?

In Azure **Service Bus**, message **sessions** enable the grouping of related messages for **ordered** processing. By using sessions, messages with the same **session ID** are processed **sequentially** by the same consumer, maintaining the order and allowing **stateful** processing across multiple related messages. This is useful for scenarios where message **order** and **context** need to be preserved.